## Création du Dataloader avec pytorch 

In [1]:
import numpy as np
import pandas as pd
import csv
import pickle
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from matplotlib.pyplot import matplotlib
import os

In [3]:
%reload_ext autoreload
%autoreload
%matplotlib inline

In [2]:
chemin = "O:\SU_March_Valeurs\Centre d'Excellence en Dérivés\Lab Fintech\SEDAR - Informations matérielles"
os.chdir(chemin)


##     Liste des fonctions et classes utilisees

In [24]:
## extraction de la liste des tickers dans le fichiers des rendements
def extraction_ticker( listes_columns):
    tickers= []
    listes_columns= listes_columns[0:58]
    for col in listes_columns:
        col= col.split()[0]
        if '/' in col:
            col=col.split('/')[0]
        elif '.' in col:
            col=col.split('.')[0]
        tickers.append(col)
    return tickers



## fonction qui permet d'avoir les données sous une forme plus simplifiée 
def data_price_transform(df, save= False):
    df_new={'dates':[], 'stock_symbol':[], 'prix':[]}
    df_new = pd.DataFrame(data = df_new, index=None)
    stocksymbols=list(df.columns)[1:]
    for i in range(len(df)):
        prix= list(df.iloc[i].values[1:])
        dates=[df['Dates: '][i]]*(len(df.columns)-1)
        for date, stocksymbol,price in zip(dates, stocksymbols, prix):
            df_new=df_new.append({'dates':date, 'stock_symbol':stocksymbol, 'prix':price}, ignore_index=True)
        if save:
            df_new.to_csv('data_transform.csv')
    return df_new


## fonction qui prend en entree un index et teste si la valeurs du stoctsymbol dans le metadada est contenue
#dans la liste des tickers du tsx60
def test(metadata, tickers, index):
    if metadata['STOCKSYMBOL'][index] in tickers:
        return index 

    
    
class DatasetNew(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_pickle(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        stocksymbol = self.data.iloc[index,1:]['STOCKSYMBOL']
        X = self.data.iloc[index, 1:] # 
        #X=list(X)
        y = self.data.iloc[index ,1:]['r {} CN Equity'.format(stocksymbol)] #.values
        if self.transform is not None:
            X = self.transform(X)
        return X,y 

In [6]:
price = pd.read_excel('tsx60 rendement price volume.xlsx')
metadata = pd.read_pickle('metadata.pkl')

## Modification du dataset du rendement

In [73]:
# TODO Le format doit être retravaillé.
#   Chaque ligne doit correspondre à une observation.
#   Pour être plus clair: chaque ligne doit être: Date, Stock symbol, Prix

df1 = pd.read_csv('TSX60 rendement.csv')
df_new = df1.iloc[:,58:]
df=pd.concat([ df1['Dates: '], df_new], axis =1)
df.head(7)

,Dates:,r AEM CN Equity,r ATD/B CN Equity,r BCE CN Equity,r BMO CN Equity,r BNS CN Equity,r ABX CN Equity,r BHC CN Equity,r BB CN Equity,r BBD/B CN Equity,...,r SLF CN Equity,r SU CN Equity,r TRP CN Equity,r T CN Equity,r TECK/B CN Equity,r TRI CN Equity,r TD CN Equity,r WCN CN Equity,r WPM CN Equity,r CCO.CN Equity
0,2020-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-12,0.004854,0.009350,0.004832,0.003047,-0.000669,-0.029061,0.040143,0.012563,-0.059880,...,0.000613,0.001022,-0.000677,0.015038,0.008082,-0.002863,0.005069,-0.026613,-0.005716,0.018608
2,2020-02-11,0.006483,0.011975,-0.003413,0.000980,-0.004823,0.005740,-0.010045,0.022333,-0.095541,...,-0.006278,-0.001277,-0.004739,-0.004517,-0.025120,0.001667,0.003318,-0.002034,0.021949,-0.027006
3,2020-02-10,-0.004041,0.025675,0.002335,-0.000098,0.000942,0.001223,0.002670,-0.019417,0.056338,...,-0.008475,-0.000256,0.000816,-0.000726,-0.038377,0.007768,-0.000265,0.004453,-0.010483,-0.043265
4,2020-02-07,-0.019909,-0.007401,-0.002640,0.003328,0.000000,-0.014251,-0.003728,-0.008663,-0.020000,...,-0.003885,-0.010997,-0.004214,-0.009444,0.013683,-0.008258,0.002249,-0.004508,-0.018863,-0.025597
5,2020-02-06,0.025230,0.007895,-0.000311,-0.000195,-0.004169,0.008674,0.027533,0.021223,-0.027211,...,-0.006240,0.021205,-0.001229,-0.008984,0.034308,0.006939,-0.000264,0.005585,0.017646,0.006130
6,2020-02-05,-0.000379,-0.025892,-0.020717,-0.006246,-0.009995,-0.019247,0.003642,0.008557,0.048951,...,-0.001413,0.038238,-0.006971,-0.011286,0.033714,-0.019572,-0.006734,-0.005329,-0.021480,0.014795


In [64]:
df_new= data_price_transform(df)

In [72]:
df_new= df_new.dropna()
df_new.head(20)

,dates,stock_symbol,prix
59,2020-02-12,r AEM CN Equity,0.004854
60,2020-02-12,r ATD/B CN Equity,0.009350
61,2020-02-12,r BCE CN Equity,0.004832
62,2020-02-12,r BMO CN Equity,0.003047
63,2020-02-12,r BNS CN Equity,-0.000669
64,2020-02-12,r ABX CN Equity,-0.029061
65,2020-02-12,r BHC CN Equity,0.040143
66,2020-02-12,r BB CN Equity,0.012563
67,2020-02-12,r BBD/B CN Equity,-0.059880
68,2020-02-12,r BAM/A CN Equity,-0.030762


In [70]:
df_new.to_csv('tsx60 rendement new.csv')

## Creation du dataset


1. Merge price and metadata by date

In [16]:
%%time
## Merger les rendements avec les donnees du metadata suivant la colonne des dates
price['date']= [str(date.date()) for date in price['Dates: ']]
metadata['date'] = [str(date.date()) for date in metadata['FILING_DATE']]
data_merge = metadata.merge(price, on = 'date')
#data_merge.to_csv('data_test.csv')
data_merge.shape

Wall time: 574 ms


In [11]:
%%time
mapping = metadata[["FILING_DATE", "STOCKSYMBOL"]]
# TODO rajouter la colonne manquante symbol stock à prices
# TODO renammer les colonnes (ex: 'Dates: ' n'est pas terrile avec l'espace)
mapping = pd.merge(mapping, price["Dates: "], left_on="FILING_DATE", right_on="Dates: ")
mapping.shape

Wall time: 4.99 ms


In [12]:
mapping.sample(5)

NameError: name 'mapping' is not defined

In [18]:
## enregistrer le dataset. 
#with open('data_test.pkl', 'wb') as f:
#    pickle.dump(data_merge, f)

2. Extraction des donnees du data_merge  donc les stacksymbols sont dans le TSX60

In [19]:
#liste des stack symbols non nan du metadata
stocksymbol = [elt for elt in list(set(metadata['STOCKSYMBOL'])) if str(elt)!='nan']
#liste des tickers du TSX60 conforment aux stcksymbol dans le metadata
#tickers = extraction_ticker(price.columns)
# data du TSX60 contenue dans le metadata

# TODO Ce code est très lent
#tsx60_metadata= data_merge.iloc[[test(data_merge, tickers, index) for index in range(len(data_merge)) if test(data_merge, tickers, index)!=None]]
#with open('tsx60_metadata.pkl', 'wb') as f:
#    pickle.dump(tsx60_metadata, f)

3. Utilisation du dataloader de pytoch pour construire le dataset

In [20]:
train_data= DatasetNew('tsx60_metadata.pkl', transform=None)

In [21]:
X, y = train_data.__getitem__(5)
len(train_data)
#train_loader=DataLoader(train_data, batch_size=8, shuffle = True)
#X, y

1638

In [344]:
train_loader=DataLoader(train_data, batch_size=8, shuffle = True)

In [347]:
X

['02784774-00000153-00002236-i@#Soco#Sedar#2018_UR#NBC_Pricing_Suppl_July3_2018#October_2018#Oct1_ACCI627_MM_FR-PDF',
 '02784774',
 '00000153',
 '002',
 'Other Issuers',
 '001',
 'Securities Offerings',
 '001082',
 '005036',
 'Marketing materials - French',
 Timestamp('2018-10-01 00:00:00'),
 Timestamp('2018-10-01 00:00:00'),
 'NF ',
 '\n',
 '\n',
 '\n',
 '\n',
 WindowsPath('2018-10-01/02784774-00000153-00002236-i@#Soco#Sedar#2018_UR#NBC_Pricing_Suppl_July3_2018#October_2018#Oct1_ACCI627_MM_FR-PDF-DDS.pkl'),
 nan,
 nan,
 nan,
 nan,
 'OTHER_ISSUER',
 '00002236',
 'National Bank of Canada',
 'Banque Nationale du Canada',
 'Dominic Paradis, Vice-président, Affaires juridiques et secrétaire corporatif',
 '514',
 '394-6751',
 None,
 '514',
 '394-8434',
 Timestamp('2015-02-04 17:47:49'),
 '\n',
 'Canada',
 '633067',
 '099199000000000000000090000090990099999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999',
 '004',
 Timestamp('185

In [251]:
# Aplication 


In [316]:
tsx60_metadata.iloc[1, 1:]['r CM CN Equity']

0.663

In [291]:
price.columns[60:100]

Index(['r BCE CN Equity ', 'r BMO  CN Equity  ', 'r BNS CN Equity',
       'r ABX CN Equity', 'r BHC CN Equity', 'r BB CN Equity',
       'r BBD/B CN Equity', 'r BAM/A CN Equity', 'r BIP CN Equity',
       'r CCL/B CN Equity', 'r GIB/A CN Equity', 'r CCO CN Equity',
       'r CM CN Equity', 'r CNR CN Equity', 'r CNQ CN Equity',
       'r CP CN Equity', 'r CTC/A  CN Equity', 'r WEED CN Equity',
       'r CVE CN Equity', 'r CSU CN Equity', 'r DOL CN Equity',
       'r EMA CN Equity', 'r ENB CN Equity', 'r FM CN Equity',
       'r FST CN Equity', 'r FNV CN Equity', 'r WN CN Equity',
       'r GIL CN Equity', 'r IMO CN Equity', 'r IPL CN Equity',
       'r K CN Equity', 'r KL CN Equity', 'r L  CN Equity', 'r MG CN Equity',
       'r MFC CN Equity', 'r MRU CN Equity', 'r NA CN Equity',
       'r NTR CN Equity', 'r OTEX CN Equity', 'r PPL CN Equity'],
      dtype='object')

In [341]:
tsx60_metadata.columns

Index(['CTL_INFO', 'ACCESSION_NO', 'PROJECT_NO', 'SUBMISSION_NO', 'FILER_TYPE',
       'FILER_DESC', 'FILING_TYPE', 'FILING_DESC', 'DOCUMENT_TYPE',
       'DOCUMENT_SUBTYPE',
       ...
       'v SLF CN Equity', 'v SU CN Equity', 'v TRP CN Equity', 'v T CN Equity',
       'v TECK/B CN Equity', 'v TRI CN Equity', 'v TD CN Equity',
       'v WCN CN Equity', 'v WPM CN Equity', 'v CCO.CN Equity'],
      dtype='object', length=324)